Q2.

In [486]:
import numpy as np
import pandas as pd 

#load the heart dataset 
df = pd.read_csv('Heart.csv')
df.head()

Unnamed: 0  Age  Sex     ChestPain  RestBP  Chol  Fbs  RestECG  MaxHR  \
0           1   63    1       typical     145   233    1        2    150   
1           2   67    1  asymptomatic     160   286    0        2    108   
2           3   67    1  asymptomatic     120   229    0        2    129   
3           4   37    1    nonanginal     130   250    0        0    187   
4           5   41    0    nontypical     130   204    0        2    172   

   ExAng  Oldpeak  Slope   Ca        Thal  AHD  
0      0      2.3      3  0.0       fixed   No  
1      1      1.5      2  3.0      normal  Yes  
2      1      2.6      2  2.0  reversable  Yes  
3      0      3.5      3  0.0      normal   No  
4      0      1.4      1  0.0      normal   No

In [487]:
# remove nan 
df = df.dropna()

# drop the Unnamed column
df = df.drop(['Unnamed: 0'], axis=1)

In [488]:
print(df.shape)

(297, 14)


In [489]:
#chest pain , thal are string values so we need to convert them to numerical values 

# print the unique values of chest pain
print(df.ChestPain.unique())

# print the unique values of thal
print(df.Thal.unique())

['typical' 'asymptomatic' 'nonanginal' 'nontypical']
['fixed' 'normal' 'reversable']


In [490]:
# encode string values to numerical values
df['ChestPain'] = df['ChestPain'].map({'typical':0, 'asymptomatic':1, 'nonanginal':2, 'nontypical':3})

df['Thal'] = df['Thal'].map({'fixed':0, 'normal':1, 'reversable':2})

df_ = df.copy()

In [491]:
df.head()

Age  Sex  ChestPain  RestBP  Chol  Fbs  RestECG  MaxHR  ExAng  Oldpeak  \
0   63    1          0     145   233    1        2    150      0      2.3   
1   67    1          1     160   286    0        2    108      1      1.5   
2   67    1          1     120   229    0        2    129      1      2.6   
3   37    1          2     130   250    0        0    187      0      3.5   
4   41    0          3     130   204    0        2    172      0      1.4   

   Slope   Ca  Thal  AHD  
0      3  0.0     0   No  
1      2  3.0     1  Yes  
2      2  2.0     2  Yes  
3      3  0.0     1   No  
4      1  0.0     1   No

In [492]:
# seperate the data into 2 classes ie: with AHD = No and with AHD = Yes
df_no = df[df.AHD == 'No']
df_yes = df[df.AHD == 'Yes']

# remove the AHD column from the data and store it in a separate variable
target = df['AHD']
df_no = df_no.drop(['AHD'], axis=1)
df_yes = df_yes.drop(['AHD'], axis=1)
df_no_fda_pca = df_no.copy()
df_yes_fda_pca = df_yes.copy()

# print the shape of the data
print(df_no.shape)
print(df_yes.shape)

(160, 13)
(137, 13)


In [493]:
df_no.head()

Age  Sex  ChestPain  RestBP  Chol  Fbs  RestECG  MaxHR  ExAng  Oldpeak  \
0   63    1          0     145   233    1        2    150      0      2.3   
3   37    1          2     130   250    0        0    187      0      3.5   
4   41    0          3     130   204    0        2    172      0      1.4   
5   56    1          3     120   236    0        0    178      0      0.8   
7   57    0          1     120   354    0        0    163      1      0.6   

   Slope   Ca  Thal  
0      3  0.0     0  
3      3  0.0     1  
4      1  0.0     1  
5      1  0.0     1  
7      1  0.0     1

In [494]:
df_yes.head()

Age  Sex  ChestPain  RestBP  Chol  Fbs  RestECG  MaxHR  ExAng  Oldpeak  \
1   67    1          1     160   286    0        2    108      1      1.5   
2   67    1          1     120   229    0        2    129      1      2.6   
6   62    0          1     140   268    0        2    160      0      3.6   
8   63    1          1     130   254    0        2    147      0      1.4   
9   53    1          1     140   203    1        2    155      1      3.1   

   Slope   Ca  Thal  
1      2  3.0     1  
2      2  2.0     2  
6      3  2.0     1  
8      2  1.0     2  
9      3  0.0     2

In [495]:
# applying FDA on the data
m0 = df_no.mean()
m1 = df_yes.mean()

#convert to arrays and realise the data 
m0 = np.array(m0)
m1 = np.array(m1)

print(m0)
print(m0.shape)
print(m1)
print(m1.shape)


[5.2643750e+01 5.5625000e-01 1.8062500e+00 1.2917500e+02 2.4349375e+02
 1.4375000e-01 8.4375000e-01 1.5858125e+02 1.4375000e-01 5.9875000e-01
 1.4125000e+00 2.7500000e-01 1.1312500e+00]
(13,)
[5.67591241e+01 8.17518248e-01 1.21167883e+00 1.34635036e+02
 2.51854015e+02 1.45985401e-01 1.17518248e+00 1.39109489e+02
 5.40145985e-01 1.58905109e+00 1.82481752e+00 1.14598540e+00
 1.55474453e+00]
(13,)


In [496]:
# finding So and S1 
So = (1/(len(df_no)-1)) * np.dot((df_no-m0).T, (df_no-m0))
S1 = (1/(len(df_yes)-1)) * np.dot((df_yes-m1).T, (df_yes-m1))

# print shape of So and S1
print(So.shape)
print(S1.shape)

(13, 13)
(13, 13)


In [497]:
# finding eigenvector 
n0 = len(df_no)
n1 = len(df_yes)
e = np.linalg.inv(n0*So + n1*S1) @ (m0 - m1)
print(e)
print(e.shape)  

[ 5.32531386e-05 -4.82219162e-03  1.17995809e-03 -5.78858517e-05
 -1.02315676e-05  2.60264614e-03 -7.68458328e-04  9.82449407e-05
 -4.86936172e-03 -8.35583529e-04 -2.19309978e-03 -4.43568393e-03
 -4.11557527e-03]
(13,)


In [498]:
# finding the projection of the data on the eigenvector
df_no = df_no @ e
df_yes = df_yes @ e

print(df_no.shape)
print(df_yes.shape)

(160,)
(137,)


In [499]:
df_no = pd.DataFrame(df_no)
df_yes = pd.DataFrame(df_yes)

print(df_no.shape)
df_no.head()


(160, 1)


0
0 -0.004943
3 -0.005823
4  0.003994
5  0.002849
7 -0.002018

In [500]:
print(df_yes.shape)
df_yes.head()

(137, 1)


0
1 -0.031120
2 -0.026758
6 -0.014756
8 -0.015729
9 -0.016977

In [501]:
# combining df_no and df_yes

df = pd.concat([df_no, df_yes], axis=0)
df = df.sort_index()
print(df.shape)
df.head()

(297, 1)


0
0 -0.004943
1 -0.031120
2 -0.026758
3 -0.005823
4  0.003994

In [502]:
# add the target column to the data
df['AHD'] = target

In [503]:
df.head()

0  AHD
0 -0.004943   No
1 -0.031120  Yes
2 -0.026758  Yes
3 -0.005823   No
4  0.003994   No

In [504]:
# rename the colum 0 
df = df.rename(columns={0:'F1'})
df.head()

F1  AHD
0 -0.004943   No
1 -0.031120  Yes
2 -0.026758  Yes
3 -0.005823   No
4  0.003994   No

In [505]:
df_fda = df

#replacing the target values with 0 and 1
df_fda['AHD'] = df_fda['AHD'].map({'No':0, 'Yes':1})

df_fda.head()

#No:0
#Yes:1

F1  AHD
0 -0.004943    0
1 -0.031120    1
2 -0.026758    1
3 -0.005823    0
4  0.003994    0

In [506]:
# applying logistic regression on the data 

#splitting the data into train and test
from sklearn.model_selection import train_test_split
# state = 5
X_train, X_test, y_train, y_test = train_test_split(df.drop('AHD', axis=1), df['AHD'], test_size=0.3, random_state=state)

# store the indexes to doing similar splitting in PCA + FDA 
X_train_index = X_train.index
X_test_index = X_test.index

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(207, 1)
(90, 1)
(207,)
(90,)


In [507]:
#applying logistic regression to train on the X_train and y_train and predict values of X_test
# we will be using implementation by scratch 
print(X_train.shape)

# adding another column of 1s to the X_train to account for the bias term
X_train = np.c_[np.ones((X_train.shape[0], 1)), X_train]
weight = np.zeros(X_train.shape[1])

print(X_train.shape)

(207, 1)
(207, 2)


In [508]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# applying gradient descent
learning_rate = 0.1
iterations = 500000
for i in range(iterations):
    weight = weight - (learning_rate/len(X_train)) * np.dot(X_train.T, (sigmoid(np.dot(X_train, weight)) - y_train))

print(weight)

KeyboardInterrupt: 

In [ ]:
X_test.head()

F1
56  -0.011828
118 -0.027260
133 -0.006142
24  -0.027012
265 -0.010980

In [ ]:
# adding another column of 1s to the X_test to account for the bias term
X_test = np.c_[np.ones((X_test.shape[0], 1)), X_test]

# predicting the values of X_test
y_pred = sigmoid(np.dot(X_test, weight))

# converting the predicted values to 0 and 1
for i in range(len(y_pred)):
    if y_pred[i] > 0.5:
        y_pred[i] = 1
    else:
        y_pred[i] = 0

print(y_pred)

# calculating the accuracy of the model
correct = 0
for i in range(len(y_pred)):
    if y_pred[i] == y_test.values[i]:
        correct += 1

ans1 = (correct/len(y_pred)) * 100
print("ACCURACY% WITH FDA ONLY: ", (correct/len(y_pred)) * 100)

[1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.
 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0.
 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1.]
ACCURACY% WITH FDA ONLY:  83.33333333333334


In [ ]:
print(df_fda.shape)
df_yes_fda_pca.head()

(297, 2)


Age  Sex  ChestPain  RestBP  Chol  Fbs  RestECG  MaxHR  ExAng  Oldpeak  \
1   67    1          1     160   286    0        2    108      1      1.5   
2   67    1          1     120   229    0        2    129      1      2.6   
6   62    0          1     140   268    0        2    160      0      3.6   
8   63    1          1     130   254    0        2    147      0      1.4   
9   53    1          1     140   203    1        2    155      1      3.1   

   Slope   Ca  Thal  
1      2  3.0     1  
2      2  2.0     2  
6      3  2.0     1  
8      2  1.0     2  
9      3  0.0     2

In [ ]:
df_no_fda_pca.head()

Age  Sex  ChestPain  RestBP  Chol  Fbs  RestECG  MaxHR  ExAng  Oldpeak  \
0   63    1          0     145   233    1        2    150      0      2.3   
3   37    1          2     130   250    0        0    187      0      3.5   
4   41    0          3     130   204    0        2    172      0      1.4   
5   56    1          3     120   236    0        0    178      0      0.8   
7   57    0          1     120   354    0        0    163      1      0.6   

   Slope   Ca  Thal  
0      3  0.0     0  
3      3  0.0     1  
4      1  0.0     1  
5      1  0.0     1  
7      1  0.0     1

In [ ]:
# Now i will be applying PCA and then FDA on the data

# applying PCA on the data
# PCA will be applied using sklearn since it has been alrady implemented in the previous assignment

# combining df_no_fda_pca and df_yes_fda_pca 
df_fda_pca = pd.concat([df_no_fda_pca, df_yes_fda_pca], axis=0)

# applying PCA on the data
from sklearn.decomposition import PCA
pca = PCA(n_components=4)
df_fda_pca = pca.fit_transform(df_fda_pca)
df_fda_pca = pd.DataFrame(df_fda_pca)
df_fda_pca.head()

0          1          2          3
0  -13.323820   2.934904  14.985609   6.689009
1    1.830973 -39.888045   1.243501 -10.801803
2  -43.892777 -23.911165   2.152961  -8.038069
3  -11.933703 -28.679730  -6.687208   8.142569
4  105.896429 -15.558412 -15.134009   3.320383

In [ ]:
# now applying FDA on the data

# adding the target column to the data
df_fda_pca['AHD'] = target

# replacing the target values with 0 and 1
df_fda_pca['AHD'] = df_fda_pca['AHD'].map({'No':0, 'Yes':1}) # type: ignore

print(df_fda_pca.shape)
print(df_fda_pca.head()) # type: ignore

(297, 5)
            0          1          2          3  AHD
0  -13.323820   2.934904  14.985609   6.689009  0.0
1    1.830973 -39.888045   1.243501 -10.801803  1.0
2  -43.892777 -23.911165   2.152961  -8.038069  1.0
3  -11.933703 -28.679730  -6.687208   8.142569  0.0
4  105.896429 -15.558412 -15.134009   3.320383  0.0


In [ ]:
# dividing the data into 2 classes to apply AHD = 0 and AHD = 1

df_no_fda_pca = df_fda_pca[df_fda_pca['AHD'] == 0]
df_yes_fda_pca = df_fda_pca[df_fda_pca['AHD'] == 1]

print(df_no_fda_pca.shape)
print(df_yes_fda_pca.shape)

(160, 5)
(132, 5)


In [ ]:
# saving and dropping the target column
target = df_fda_pca['AHD']
df_no_fda_pca = df_no_fda_pca.drop('AHD', axis=1) # type: ignore
df_yes_fda_pca = df_yes_fda_pca.drop('AHD', axis=1) # type: ignore

print(df_no_fda_pca.shape)
print(df_yes_fda_pca.shape)

(160, 4)
(132, 4)


In [ ]:
df_no_fda_pca.head()

0          1          2         3
0  -13.323820   2.934904  14.985609  6.689009
3  -11.933703 -28.679730  -6.687208  8.142569
4  105.896429 -15.558412 -15.134009  3.320383
5  -54.725067   3.755534  11.253582  2.419415
7   14.575842 -26.387146 -10.394245 -5.127718

In [ ]:
df_yes_fda_pca.head()

0          1          2          3
1   1.830973 -39.888045   1.243501 -10.801803
2 -43.892777 -23.911165   2.152961  -8.038069
6  47.031760  -2.794346   6.245926  -0.604148
8 -46.308035  -7.243398  43.575097  -5.172687
9 -78.211398 -20.114167  26.031327   5.855955

In [ ]:
# applying FDA on the data

# finding the mean of the data as applied in the previous assignment
m0 = df_no_fda_pca.mean()
m1 = df_yes_fda_pca.mean()

n0 = df_no_fda_pca.shape[0]
n1 = df_yes_fda_pca.shape[0]

So = (1/(n0-1)) * np.dot((df_no_fda_pca - m0).T, (df_no_fda_pca - m0))
S1 = (1/(n1-1)) * np.dot((df_yes_fda_pca - m1).T, (df_yes_fda_pca - m1))

print(So.shape)
print(S1.shape)
print(So)
print(S1)

(4, 4)
(4, 4)
[[3242.31015694 -141.24001527   51.99728211   -6.33048731]
 [-141.24001527  509.20246855   14.63919066  -22.58228268]
 [  51.99728211   14.63919066  334.22951361   -5.46091965]
 [  -6.33048731  -22.58228268   -5.46091965   64.81036206]]
[[2166.27769273  164.29215184  -67.71743105   11.49538825]
 [ 164.29215184  604.35298681  -25.26455757   28.60646125]
 [ -67.71743105  -25.26455757  287.93482393    5.83478902]
 [  11.49538825   28.60646125    5.83478902   49.79078461]]


In [ ]:
print(m0.shape)
print(m1.shape)

(4,)
(4,)


In [ ]:
e = np.dot(np.linalg.inv(So + S1), (m0 - m1))
e

array([6.73760874e-04, 1.60231201e-03, 2.85788104e-03, 7.43669125e-05])

In [ ]:
# finding the projection of the data on the eigenvector
df_no_fda_pca = df_no_fda_pca @ e
df_yes_fda_pca = df_yes_fda_pca @ e

print(df_no_fda_pca.shape)
print(df_yes_fda_pca.shape)


(160,)
(132,)


In [ ]:
df_no_fda_pca.head()

0    0.039050
3   -0.072500
4    0.003415
5    0.001487
7   -0.062547
dtype: float64

In [ ]:
df_yes_fda_pca.head()

1   -0.059929
2   -0.062331
6    0.045016
8    0.081341
9   -0.010095
dtype: float64

In [ ]:
df_no_fda_pca = pd.DataFrame(df_no)
df_yes_fda_pca = pd.DataFrame(df_yes)

In [ ]:
df_no_fda_pca.head()


0
0 -0.004943
3 -0.005823
4  0.003994
5  0.002849
7 -0.002018

In [ ]:
df_yes_fda_pca.head()


0
1 -0.031120
2 -0.026758
6 -0.014756
8 -0.015729
9 -0.016977

In [ ]:
# combining the data
df_fda_pca = pd.concat([df_no_fda_pca, df_yes_fda_pca], axis=0)

# sorting the data
df_fda_pca = df_fda_pca.sort_index()
# adding the target column
df_fda_pca['AHD'] = target
# cleaning the data
df_fda_pca = df_fda_pca.dropna()
df_fda_pca.head()

0  AHD
0 -0.004943  0.0
1 -0.031120  1.0
2 -0.026758  1.0
3 -0.005823  0.0
4  0.003994  0.0

In [ ]:
# splitting the data into train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_fda_pca.drop('AHD', axis=1), df_fda_pca['AHD'], test_size=0.3, random_state=state)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(204, 1)
(88, 1)
(204,)
(88,)


In [ ]:
# now we will use the logistic regression model in order to learn from X_train and y_train and find the weight vector 
# and then we will use the weight vector to predict the values of X_test and then we will calculate the accuracy of the model

X_train = np.c_[np.ones((X_train.shape[0], 1)), X_train]
weight = np.zeros(X_train.shape[1])

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# applying gradient descent
learning_rate = 0.1
iterations = 500000
for i in range(iterations):
    weight = weight - (learning_rate/len(X_train)) * np.dot(X_train.T, (sigmoid(np.dot(X_train, weight)) - y_train))

print(weight)

[  -1.17920423 -100.9342512 ]


In [ ]:
# adding another column of 1s to the X_test to account for the bias term
X_test = np.c_[np.ones((X_test.shape[0], 1)), X_test]

# predicting the values of X_test
y_pred = sigmoid(np.dot(X_test, weight))

print(y_pred)

# converting the predicted values to 0 and 1
for i in range(len(y_pred)):
    if y_pred[i] > 0.5:
        y_pred[i] = 1
    else:
        y_pred[i] = 0

print(y_pred)

# calculating the accuracy of the model
correct = 0
for i in range(len(y_pred)):
    if y_pred[i] == y_test.values[i]:
        correct += 1

ans2 = (correct/len(y_pred)) * 100
print("ACCURACY% WITH FDA ONLY: ", (correct/len(y_pred)) * 100)

[0.26295931 0.23945563 0.49430087 0.44092489 0.29714235 0.4026002
 0.41254859 0.19502722 0.18184073 0.71739719 0.30855773 0.50314229
 0.30603476 0.49821657 0.23373876 0.82451317 0.38783368 0.23141866
 0.33476302 0.70510608 0.63162156 0.23238754 0.17028736 0.2067716
 0.3832337  0.86999238 0.21189385 0.29297178 0.14904253 0.11809865
 0.7227674  0.22081122 0.69460736 0.11700806 0.26929967 0.2226331
 0.50366277 0.37404245 0.67461815 0.68963298 0.33358798 0.20416328
 0.2915853  0.77576924 0.21764404 0.47238001 0.17940556 0.54476318
 0.21161712 0.3291709  0.25698962 0.49005062 0.35355221 0.22396686
 0.15652931 0.42104442 0.37969122 0.20873311 0.88052193 0.1507088
 0.82503258 0.34279064 0.71884217 0.3943142  0.5769418  0.21230709
 0.54238662 0.82809909 0.62726365 0.63076099 0.62446292 0.53188197
 0.58026185 0.27448475 0.18708096 0.82076604 0.69072973 0.60281657
 0.35628916 0.79736087 0.61393775 0.2500822  0.53433426 0.58501693
 0.69647648 0.23504279 0.48227129 0.52510315]
[0. 0. 0. 0. 0. 0. 0

In [ ]:
# SUMMARY 
print("ACCURACY% WITH FDA ONLY: ", ans1)
print("ACCURACY% WITH FDA AND PCA: ", ans2)

ACCURACY% WITH FDA ONLY:  83.33333333333334
ACCURACY% WITH FDA AND PCA:  86.36363636363636


In [ ]:
state = 5

In [510]:
# k fold cross validation for FDA vs FDA + PCA for k = 5

# state = 0
fda  = 83.33333333333334
fda_pca = 84.09
# state = 1
fda  = 77.7
fda_pca = 79.5
# state = 2
fda  = 85.55
fda_pca = 85.22
# state = 3
fda  = 83.33
fda_pca = 81.82  
# state = 4
fda  = 86.6
fda_pca = 80.6
# state = 5
fda  = 83.33
fda_pca = 86.36

print("FDA: ", (83 + 77 + 85 + 83 + 86 + 83) / 6)
print("FDA + PCA: ", (84 + 79 + 85 + 81 + 80 + 86) / 6)


FDA:  82.83333333333333
FDA + PCA:  82.5


The accuracy with FDA seems to be more than FDA+PCA after testing multiple times and k cross validation 
However in particular cases the order is inverted, hence I think there is no correct answer to it and it depends on the data , number of features and 
splitting of the data
Both seem to be almost identical though 
since fda+pca is more than fda more number of times 

hence mean and mode are contradictory 
-> inconclusive 

But if at all I need to say 1 and based upon my testing: 
it is FDA only 